<a href="https://colab.research.google.com/github/joyalkenus/My-website/blob/master/Project_ImageSearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [60]:
#@title Import dependencies
import ipywidgets as widgets
from IPython.display import clear_output, display, Image
import os


In [61]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [62]:
#@title Clone the repository of Recognise anything Model
!git clone https://github.com/xinyu1205/recognize-anything.git

%cd recognize-anything
!git checkout ec6b4241c5036e337a4543838deb9bff4990de97

Cloning into 'recognize-anything'...
remote: Enumerating objects: 669, done.
remote: Counting objects: 100% (379/379), done.
remote: Compressing objects: 100% (187/187), done.
remote: Total 669 (delta 264), reused 240 (delta 192), pack-reused 290
Receiving objects: 100% (669/669), 27.12 MiB | 27.44 MiB/s, done.
Resolving deltas: 100% (351/351), done.
/content/recognize-anything/recognize-anything/recognize-anything/recognize-anything
Note: switching to 'ec6b4241c5036e337a4543838deb9bff4990de97'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detac

In [63]:
!cp -r "/content/drive/My Drive/ImageSample" "/content/recognize-anything/"



In [64]:
!ls "/content/recognize-anything/ImageSample"


1.jpg  2.jpg


In [65]:
#@title Install dependencies
!pip install timm transformers fairscale pycocoevalcap

clear_output()

In [66]:
# Download checkpoints
model_widget = widgets.Dropdown(
    options=["RAM", "Tag2Text"],
    value="RAM",
    description="Select mdoel:"
)
display(model_widget)

Dropdown(description='Select mdoel:', options=('RAM', 'Tag2Text'), value='RAM')

In [67]:
model = model_widget.value

In [68]:
def download_checkpoints(model):
    print('You selected', model)
    if not os.path.exists('pretrained'):
        os.makedirs('pretrained')

    if model == "RAM":
        ram_weights_path = 'pretrained/ram_swin_large_14m.pth'
        if not os.path.exists(ram_weights_path):
            !wget https://huggingface.co/spaces/xinyu1205/Recognize_Anything-Tag2Text/resolve/main/ram_swin_large_14m.pth -O pretrained/ram_swin_large_14m.pth
        else:
            print("RAM weights already downloaded!")
    else:
        tag2text_weights_path = 'pretrained/tag2text_swin_14m.pth'
        if not os.path.exists(tag2text_weights_path):
            !wget https://huggingface.co/spaces/xinyu1205/Recognize_Anything-Tag2Text/resolve/main/tag2text_swin_14m.pth -O pretrained/tag2text_swin_14m.pth
        else:
            print("Tag2Text weights already downloaded!")

download_checkpoints(model)
print(model, 'weights are downloaded!')

You selected RAM
--2024-01-26 03:00:43--  https://huggingface.co/spaces/xinyu1205/Recognize_Anything-Tag2Text/resolve/main/ram_swin_large_14m.pth
Resolving huggingface.co (huggingface.co)... 65.8.178.93, 65.8.178.12, 65.8.178.27, ...
Connecting to huggingface.co (huggingface.co)|65.8.178.93|:443... connected.
HTTP request sent, awaiting response... 307 Temporary Redirect
Location: /spaces/xinyu1205/recognize-anything/resolve/main/ram_swin_large_14m.pth [following]
--2024-01-26 03:00:43--  https://huggingface.co/spaces/xinyu1205/recognize-anything/resolve/main/ram_swin_large_14m.pth
Reusing existing connection to huggingface.co:443.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/e6/78/e678f8565485a3f321b1180e4c7e1e18a89a9295028358eedffb98981b37e11a/15c729c793af28b9d107c69f85836a1356d76ea830d4714699fb62e55fcc08ed?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27ram_swin_large_14m.pth%3B+filename%3D%22ram_swin_large_14m.pt

In [77]:
images_dir_widget = widgets.Text(value="/content/drive/My Drive/ImageSample", description="Images dir:")
display(images_dir_widget)

Text(value='/content/drive/My Drive/ImageSample', description='Images dir:')

In [78]:
images_dir = images_dir_widget.value

In [90]:
image_files = [f"{images_dir}/{file}" for file in sorted(os.listdir(images_dir)) if file.lower().endswith(('.jpg', '.jpeg', '.png'))]
image_path = image_files[0]  # First image pathe_files[0]






# Create dropdown widget
image_dropdown = widgets.Dropdown(
    options=image_files,
    description='Select Image:',
)

# Create image preview widget
image_preview = widgets.Output()

# Define function to update image preview
def update_preview(change):
    global image_path
    image_path = change.new
    with image_preview:
        image_preview.clear_output()
        display(Image(filename=image_path, width=400))

# Set the initial image preview
with image_preview:
    display(Image(filename=image_files[0], width=400))

# Attach the update function to the dropdown
image_dropdown.observe(update_preview, names='value')

# Display the widgets
display(image_dropdown, image_preview)

Dropdown(description='Select Image:', options=('/content/drive/My Drive/ImageSample/1.jpg', '/content/drive/My…

Output()

In [80]:
# Define the task and run inference
task_widget = widgets.Dropdown(
    options=["one image", "multiple images"],
    value="one image",
    description="Task:"
)
display(task_widget)

Dropdown(description='Task:', options=('one image', 'multiple images'), value='one image')

In [81]:
task = task_widget.value

In [82]:
!pwd

/content/recognize-anything/recognize-anything/recognize-anything/recognize-anything


In [83]:
import os
os.listdir()

['inference_ram.py',
 'inference_ram_openset.py',
 'huggingface_web_demo.ipynb',
 '.gitignore',
 'ram',
 '.git',
 'README.md',
 'images',
 'setup.py',
 'requirements.txt',
 'setup.cfg',
 'pretrained',
 'LICENSE',
 'recognize_anything_demo.ipynb',
 'datasets',
 'batch_inference.py',
 'inference_tag2text.py']

*biggest hurdle faced*
while running was

---

code could not retrieve images from the google drive.
after a thorough closer inspection
the error was caused by the space in file path "My Drive/ImageSample" causing the parser to read it as "Drive/ImageSample"

This was new to me and learnt to be causious of spaces in the paths


In [92]:
print('You selected', model)
print('You selected', task)

def run_inference(model, task):
    global image_path
    # Ensure image_path is correctly enclosed in quotes
    image_path_quoted = f'"{image_path}"'  # Enclose in quotes
    if model == "Tag2Text" and task == "one image":
        !python inference_tag2text.py --image {image_path_quoted} \
        --pretrained pretrained/tag2text_swin_14m.pth
    elif model == "Tag2Text" and task == "multiple images":
        !python batch_inference.py --image-dir {images_dir} \
        --pretrained pretrained/tag2text_swin_14m.pth --model-type tag2text
    elif model == "RAM" and task == "one image":
        !python inference_ram.py --image {image_path_quoted} \
        --pretrained pretrained/ram_swin_large_14m.pth
    elif model == "RAM" and task == "multiple images":
        !python batch_inference.py --image-dir {images_dir} \
        --pretrained pretrained/ram_swin_large_14m.pth --model-type ram
    else:
        print('Invalid model or task')



run_inference(model, task)

You selected RAM
You selected one image
/encoder/layer/0/crossattention/self/query is tied
/encoder/layer/0/crossattention/self/key is tied
/encoder/layer/0/crossattention/self/value is tied
/encoder/layer/0/crossattention/output/dense is tied
/encoder/layer/0/crossattention/output/LayerNorm is tied
/encoder/layer/0/intermediate/dense is tied
/encoder/layer/0/output/dense is tied
/encoder/layer/0/output/LayerNorm is tied
/encoder/layer/1/crossattention/self/query is tied
/encoder/layer/1/crossattention/self/key is tied
/encoder/layer/1/crossattention/self/value is tied
/encoder/layer/1/crossattention/output/dense is tied
/encoder/layer/1/crossattention/output/LayerNorm is tied
/encoder/layer/1/intermediate/dense is tied
/encoder/layer/1/output/dense is tied
/encoder/layer/1/output/LayerNorm is tied
--------------
pretrained/ram_swin_large_14m.pth
--------------
load checkpoint from pretrained/ram_swin_large_14m.pth
vit: swin_l
Image Tags:  condiment | counter top | table | dinning tabl

In [94]:
!git push -u origin master



error: src refspec master does not match any
error: failed to push some refs to 'https://github.com/xinyu1205/recognize-anything.git'
